In [1]:
import pandas as pd
import numpy as np

In [2]:
stats_2017_1 = pd.read_csv('Data/Historic_Stats/2017_1.csv')

In [3]:
historic_stats = stats_2017_1[['Name', 'Team', 'Week', 'Opponent', 'FantasyPointsPerGame', 'FantasyPoints']]

In [4]:
historic_stats = historic_stats.rename(columns={'FantasyPointsPerGame':'FPPPG', 'FantasyPoints':'Points_Scored'})

In [5]:
historic_stats

,Name,Team,Week,Opponent,FPPPG,Points_Scored
0,Kareem Hunt,KC,1,NE,40.6,40.60
1,Alex Smith,KC,1,NE,31.0,31.02
2,Jacksonville Jaguars,JAX,1,HOU,28.0,28.00
3,Los Angeles Rams,LAR,1,IND,28.0,28.00
4,Matthew Stafford,DET,1,ARI,27.1,27.08
...,...,...,...,...,...,...
1128,Tavon Austin,LAR,1,IND,-0.2,-0.20
1129,Sean Mannion,LAR,1,IND,-0.2,-0.22
1130,Jamison Crowder,WAS,1,PHI,-0.6,-0.60
1131,Tom Savage,HOU,1,JAX,-1.5,-1.52


In [6]:
stats_list = []
for i in range(2,18):
    name = 'Data/Historic_Stats/2017_' + str(i) + '.csv'
    stats_list.append(name)

In [7]:
stats_list

['Data/Historic_Stats/2017_2.csv',
 'Data/Historic_Stats/2017_3.csv',
 'Data/Historic_Stats/2017_4.csv',
 'Data/Historic_Stats/2017_5.csv',
 'Data/Historic_Stats/2017_6.csv',
 'Data/Historic_Stats/2017_7.csv',
 'Data/Historic_Stats/2017_8.csv',
 'Data/Historic_Stats/2017_9.csv',
 'Data/Historic_Stats/2017_10.csv',
 'Data/Historic_Stats/2017_11.csv',
 'Data/Historic_Stats/2017_12.csv',
 'Data/Historic_Stats/2017_13.csv',
 'Data/Historic_Stats/2017_14.csv',
 'Data/Historic_Stats/2017_15.csv',
 'Data/Historic_Stats/2017_16.csv',
 'Data/Historic_Stats/2017_17.csv']

In [8]:
appended_data = pd.DataFrame()
for index, item in enumerate(stats_list):
    df = pd.read_csv(item)
    df = df[['Name', 'Team', 'Week', 'Opponent', 'FantasyPointsPerGameFanDuel', 'FantasyPointsFanDuel']]
    df = df.rename(columns={'FantasyPointsPerGameFanDuel':'FPPPG', 'FantasyPointsFanDuel':'Points_Scored'})
    appended_data = appended_data.append(df)


In [9]:
historic_stats = pd.concat([historic_stats, appended_data])

In [10]:
historic_stats = historic_stats.reset_index(drop=True)

In [11]:
proj_list = []
for i in range(1,18):
    name = 'Data/Historic_Proj/2017_' + str(i) + '.csv'
    proj_list.append(name)

In [12]:
historic_projections = pd.DataFrame()
for item in proj_list:
    df = pd.read_csv(item)
    df = df[['Name', 'Position', 'Week', 'Team','UpcomingOpponentPositionRank', 'FantasyPointsFanDuel', 'OperatorSalary']]
    df = df.rename(columns={'UpcomingOpponentPositionRank':'OPRNK', 'FantasyPointsFanDuel': 'Projected', 'OperatorSalary': 'Salary'})
    historic_projections = historic_projections.append(df)

In [13]:
historic_projections

,Name,Position,Week,Team,OPRNK,Projected,Salary
0,David Johnson,RB,1,ARI,8.0,20.86,9400
1,Le'Veon Bell,RB,1,PIT,31.0,21.31,9300
2,Antonio Brown,WR,1,PIT,25.0,17.17,9100
3,Julio Jones,WR,1,ATL,27.0,15.88,9000
4,Tom Brady,QB,1,NE,10.0,20.06,8900
...,...,...,...,...,...,...,...
705,San Francisco 49ers,DST,17,SF,NaN,8.20,4000
706,Dallas Cowboys,DST,17,DAL,NaN,7.70,4000
707,Tampa Bay Buccaneers,DST,17,TB,NaN,4.50,4000
708,New York Jets,DST,17,NYJ,NaN,2.90,4000


In [14]:
data_2017 = historic_stats.merge(historic_projections, how='inner', on=['Name', 'Week', 'Team'])

In [15]:
offense_2017 = data_2017[data_2017['Position'] != 'DST']

In [16]:
offense_2017 = offense_2017.drop_duplicates()

In [17]:
offense_2017['Name'].value_counts()

Chris Thompson      18
Ryan Succop         15
Samaje Perine       15
Marcus Mariota      15
Derrick Henry       15
                    ..
Jeremy McNichols     1
Bradley Marquez      1
Kyle Sloter          1
Jace Billingsley     1
Darius Powe          1
Name: Name, Length: 584, dtype: int64

In [18]:
offense_2017 = offense_2017.drop(offense_2017[(offense_2017['Name'] == 'Chris Thompson') & (offense_2017['Team'] == 'HOU')].index)

In [19]:
offense_2017['Name'].value_counts()

Samaje Perine       15
Eric Decker         15
Ryan Grant          15
Delanie Walker      15
Derrick Henry       15
                    ..
Jeremy McNichols     1
Bradley Marquez      1
Colt McCoy           1
Rashawn Scott        1
Sam Ficken           1
Name: Name, Length: 584, dtype: int64

Testing FPPG Add

In [20]:
grouped = offense_2017.groupby(['Name', 'Week']).sum()

In [21]:
grouped['Games_Played'] = 1

In [22]:
new_df = grouped.groupby(level=0).cumsum().reset_index()

In [23]:
new_df['FP_per_game'] = round(new_df['Points_Scored'] / new_df['Games_Played'], 1)

In [24]:
new_df = new_df[['Name', 'Week', 'FP_per_game', 'Games_Played']]

In [25]:
new_df['Test'] = new_df.groupby('Name')['FP_per_game'].shift()

In [26]:
new_df['Test_2'] = round(new_df.groupby('Name')['Test'].transform(lambda x: x.fillna(x.mean())), 1)

In [27]:
new_df['FPPG'] = new_df['Test_2']

In [28]:
new_df = new_df.drop(['Test', 'Test_2', 'FP_per_game'], axis=1)

In [29]:
new_df.head(50)

,Name,Week,Games_Played,FPPG
0,AJ Derby,1,1,3.3
1,AJ Derby,2,2,3.4
2,AJ Derby,3,3,1.7
3,AJ Derby,4,4,1.5
4,AJ Derby,6,5,5.0
5,AJ Derby,9,6,4.3
6,AJ Derby,10,7,3.8
7,AJ Derby,16,8,3.4
8,AJ Derby,17,9,3.2
9,AJ McCarron,4,1,0.4


In [30]:
offense_2017 = pd.merge(offense_2017, new_df, how='inner', on=['Name', 'Week'])

In [31]:
offense_2017 = offense_2017.drop('FPPPG', axis=1)

In [32]:
offense_2017[offense_2017['Name'] == 'Tom Brady']

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG
60,Tom Brady,NE,1,KC,10.68,QB,10.0,20.06,8900,1,20.8
395,Tom Brady,NE,2,NO,30.78,QB,27.0,22.17,9200,2,10.7
795,Tom Brady,NE,3,HOU,35.72,QB,5.0,18.70,9400,3,20.7
1224,Tom Brady,NE,4,CAR,20.48,QB,7.0,20.62,9500,4,25.7
1622,Tom Brady,NE,5,TB,13.62,QB,30.0,21.87,9500,5,24.4
1669,Tom Brady,NE,6,NYJ,17.18,QB,19.0,20.89,9500,6,22.3
2426,Tom Brady,NE,10,DEN,22.64,QB,15.0,18.06,8500,7,21.4
2776,Tom Brady,NE,11,LV,25.60,QB,22.0,20.53,8600,8,21.6
3139,Tom Brady,NE,12,MIA,21.68,QB,20.0,21.46,9100,9,22.1
3568,Tom Brady,NE,13,BUF,9.32,QB,7.0,19.87,9300,10,22.0


In [33]:
offense_2017['Team'].value_counts().index

Index(['TEN', 'DEN', 'NE', 'PHI', 'JAX', 'GB', 'KC', 'TB', 'WAS', 'NYG', 'ATL',
       'BUF', 'CAR', 'LV', 'LAR', 'BAL', 'CIN', 'SF', 'DAL', 'ARI', 'CHI',
       'NYJ', 'PIT', 'SEA', 'MIA', 'CLE', 'NO', 'LAC', 'HOU', 'DET', 'IND',
       'MIN'],
      dtype='object')

## Merge with NFL Betting Data

In [34]:
betting_data = pd.read_excel('Data/clean_nfl_historic_betting_data_Home.xlsx')

In [35]:
betting_data_2 = pd.read_excel('Data/clean_nfl_historic_betting_data_Away.xlsx')

In [36]:
betting_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4039 entries, 0 to 4038
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               4039 non-null   datetime64[ns]
 1   Team               4039 non-null   object        
 2   Line Close         1903 non-null   float64       
 3   Total Score Close  1903 non-null   float64       
 4   Team Proj          1903 non-null   float64       
 5   Week               1903 non-null   float64       
 6   Home / Away        1903 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 221.0+ KB


In [37]:
betting_data = pd.concat([betting_data, betting_data_2])

In [38]:
betting_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8078 entries, 0 to 4038
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               8078 non-null   datetime64[ns]
 1   Team               8078 non-null   object        
 2   Line Close         3806 non-null   float64       
 3   Total Score Close  3806 non-null   float64       
 4   Team Proj          3806 non-null   float64       
 5   Week               3806 non-null   float64       
 6   Home / Away        3806 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 504.9+ KB


In [39]:
betting_data['Date'] = betting_data['Date'].dt.year

In [40]:
betting_data = betting_data[betting_data['Date'] == 2017]

In [41]:
betting_data = betting_data[betting_data['Week'] != 0]

In [42]:
betting_data

,Date,Team,Line Close,Total Score Close,Team Proj,Week,Home / Away
846,2017,DEN,-3.0,37.5,21.75,17.0,Home
847,2017,LAC,-7.0,43.0,28.50,17.0,Home
848,2017,LAR,6.0,44.5,16.25,17.0,Home
849,2017,SEA,-8.0,38.0,27.00,17.0,Home
850,2017,ATL,-5.0,44.5,27.25,17.0,Home
...,...,...,...,...,...,...,...
1097,2017,ARI,-2.5,48.5,26.75,1.0,Away
1098,2017,JAX,5.0,38.0,14.00,1.0,Away
1099,2017,LV,2.5,49.5,22.25,1.0,Away
1100,2017,PHI,-1.0,49.5,25.75,1.0,Away


In [43]:
offense_2017 = offense_2017.merge(betting_data, how='inner', on=['Team', 'Week'])

In [44]:
offense_2017[offense_2017['Week'] == 12].head(50)

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG,Date,Line Close,Total Score Close,Team Proj,Home / Away
3124,Julio Jones,ATL,12,TB,44.80,WR,31.0,13.86,7800,9,10.0,2017,-10.5,47.0,34.00,Home
3125,Tevin Coleman,ATL,12,TB,21.70,RB,21.0,14.56,6200,9,11.3,2017,-10.5,47.0,34.00,Home
3126,Matt Ryan,ATL,12,TB,16.78,QB,23.0,17.79,7600,9,15.6,2017,-10.5,47.0,34.00,Home
3127,Matt Bryant,ATL,12,TB,11.00,K,14.0,9.00,4900,9,10.0,2017,-10.5,47.0,34.00,Home
3128,Austin Hooper,ATL,12,TB,5.30,TE,5.0,7.21,5200,9,7.2,2017,-10.5,47.0,34.00,Home
3129,Taylor Gabriel,ATL,12,TB,3.10,WR,31.0,5.19,5200,9,5.9,2017,-10.5,47.0,34.00,Home
3130,Terron Ward,ATL,12,TB,1.70,RB,21.0,6.69,4900,7,1.2,2017,-10.5,47.0,34.00,Home
3131,Derrick Coleman,ATL,12,TB,0.00,RB,21.0,0.33,4500,9,0.5,2017,-10.5,47.0,34.00,Home
3132,Levine Toilolo,ATL,12,TB,0.00,TE,5.0,1.68,4500,9,2.1,2017,-10.5,47.0,34.00,Home
3133,Eric Saubert,ATL,12,TB,0.00,TE,5.0,0.45,4500,9,0.0,2017,-10.5,47.0,34.00,Home


## 2018

In [45]:
stats_list = []
for i in range(1,18):
    name = 'Data/Historic_Stats/2018_' + str(i) + '.csv'
    stats_list.append(name)

stats_list

['Data/Historic_Stats/2018_1.csv',
 'Data/Historic_Stats/2018_2.csv',
 'Data/Historic_Stats/2018_3.csv',
 'Data/Historic_Stats/2018_4.csv',
 'Data/Historic_Stats/2018_5.csv',
 'Data/Historic_Stats/2018_6.csv',
 'Data/Historic_Stats/2018_7.csv',
 'Data/Historic_Stats/2018_8.csv',
 'Data/Historic_Stats/2018_9.csv',
 'Data/Historic_Stats/2018_10.csv',
 'Data/Historic_Stats/2018_11.csv',
 'Data/Historic_Stats/2018_12.csv',
 'Data/Historic_Stats/2018_13.csv',
 'Data/Historic_Stats/2018_14.csv',
 'Data/Historic_Stats/2018_15.csv',
 'Data/Historic_Stats/2018_16.csv',
 'Data/Historic_Stats/2018_17.csv']

In [46]:
stats_2018 = pd.DataFrame()
for index, item in enumerate(stats_list):
    df = pd.read_csv(item)
    df = df[['Name', 'Team', 'Week', 'Opponent', 'FantasyPointsPerGameFanDuel', 'FantasyPointsFanDuel']]
    df = df.rename(columns={'FantasyPointsPerGameFanDuel':'FPPPG', 'FantasyPointsFanDuel':'Points_Scored'})
    stats_2018 = stats_2018.append(df)


In [47]:
stats_2018

,Name,Team,Week,Opponent,FPPPG,Points_Scored
0,Ryan Fitzpatrick,TB,1,NO,42.3,42.28
1,Tyreek Hill,KC,1,LAC,38.8,38.80
2,Alvin Kamara,NO,1,TB,38.6,38.60
3,James Conner,PIT,1,CLE,31.7,31.70
4,Drew Brees,NO,1,TB,31.6,31.56
...,...,...,...,...,...,...
1171,Brian Hoyer,NE,17,NYJ,-0.3,-0.30
1172,Jahleel Addae,LAC,17,DEN,-2.0,-2.00
1173,Adoree' Jackson,TEN,17,IND,-2.0,-2.00
1174,Geno Smith,LAC,17,DEN,-2.2,-2.20


In [48]:
proj_list = []
for i in range(1,18):
    name = 'Data/Historic_Proj/2018_' + str(i) + '.csv'
    proj_list.append(name)
    
proj_list

['Data/Historic_Proj/2018_1.csv',
 'Data/Historic_Proj/2018_2.csv',
 'Data/Historic_Proj/2018_3.csv',
 'Data/Historic_Proj/2018_4.csv',
 'Data/Historic_Proj/2018_5.csv',
 'Data/Historic_Proj/2018_6.csv',
 'Data/Historic_Proj/2018_7.csv',
 'Data/Historic_Proj/2018_8.csv',
 'Data/Historic_Proj/2018_9.csv',
 'Data/Historic_Proj/2018_10.csv',
 'Data/Historic_Proj/2018_11.csv',
 'Data/Historic_Proj/2018_12.csv',
 'Data/Historic_Proj/2018_13.csv',
 'Data/Historic_Proj/2018_14.csv',
 'Data/Historic_Proj/2018_15.csv',
 'Data/Historic_Proj/2018_16.csv',
 'Data/Historic_Proj/2018_17.csv']

In [49]:
projections_2018 = pd.DataFrame()
for item in proj_list:
    df = pd.read_csv(item)
    df = df[['Name', 'Position', 'Week', 'Team','UpcomingOpponentPositionRank', 'FantasyPointsFanDuel', 'OperatorSalary']]
    df = df.rename(columns={'UpcomingOpponentPositionRank':'OPRNK', 'FantasyPointsFanDuel': 'Projected', 'OperatorSalary': 'Salary'})
    projections_2018 = projections_2018.append(df)

In [50]:
projections_2018

,Name,Position,Week,Team,OPRNK,Projected,Salary
0,Todd Gurley,RB,1,LAR,17.0,20.49,9000
1,Antonio Brown,WR,1,PIT,15.0,16.99,9000
2,DeAndre Hopkins,WR,1,HOU,29.0,16.25,8800
3,Aaron Rodgers,QB,1,GB,7.0,20.66,8700
4,Alvin Kamara,RB,1,NO,28.0,19.72,8700
...,...,...,...,...,...,...,...
151,Tyler Boyd,WR,17,CIN,17.0,0.00,0
152,Allen Robinson,WR,17,CHI,2.0,0.00,0
153,Kenny Golladay,WR,17,DET,28.0,0.00,0
154,Todd Gurley,RB,17,LAR,21.0,0.00,0


In [51]:
data_2018 = stats_2018.merge(projections_2018, how='inner', on=['Name', 'Week', 'Team'])

In [52]:
offense_2018 = data_2018[data_2018['Position'] != 'DST']

In [53]:
offense_2018 = offense_2018.drop_duplicates()

In [54]:
offense_2018['Name'].value_counts()

Zay Jones             16
Ben Roethlisberger    16
Larry Fitzgerald      16
Julio Jones           16
Zach Ertz             16
                      ..
Thomas Rawls           1
Daniel Carlson         1
Byron Marshall         1
Jake Elliott           1
Jerome Cunningham      1
Name: Name, Length: 576, dtype: int64

In [55]:
grouped = offense_2018.groupby(['Name', 'Week']).sum()

In [56]:
grouped['Games_Played'] = 1

In [57]:
new_df = grouped.groupby(level=0).cumsum().reset_index()

In [58]:
new_df['FP_per_game'] = round(new_df['Points_Scored'] / new_df['Games_Played'], 1)

In [59]:
new_df = new_df[['Name', 'Week', 'FP_per_game', 'Games_Played']]

In [60]:
new_df['Test'] = new_df.groupby('Name')['FP_per_game'].shift()

In [61]:
new_df['Test_2'] = round(new_df.groupby('Name')['Test'].transform(lambda x: x.fillna(x.mean())), 1)

In [62]:
new_df['FPPG'] = new_df['Test_2']

In [63]:
new_df = new_df.drop(['Test', 'Test_2', 'FP_per_game'], axis=1)

In [64]:
new_df

,Name,Week,Games_Played,FPPG
0,AJ Derby,1,1,2.6
1,AJ Derby,2,2,0.0
2,AJ Derby,3,3,4.2
3,AJ Derby,10,4,3.5
4,AJ McCarron,9,1,0.3
...,...,...,...,...
5586,Zay Jones,13,12,6.3
5587,Zay Jones,14,13,7.7
5588,Zay Jones,15,14,7.4
5589,Zay Jones,16,15,7.0


In [65]:
offense_2018 = pd.merge(offense_2018, new_df, how='inner', on=['Name', 'Week'])

In [66]:
offense_2018 = offense_2018.drop('FPPPG', axis=1)

In [67]:
offense_2018[offense_2018['Name'] == 'Tom Brady']

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG
19,Tom Brady,NE,1,HOU,22.28,QB,31.0,19.86,8600,1,18.8
426,Tom Brady,NE,2,JAX,16.36,QB,6.0,17.31,8500,2,22.3
860,Tom Brady,NE,3,DET,8.52,QB,13.0,20.49,8500,3,19.3
1155,Tom Brady,NE,4,MIA,20.96,QB,2.0,19.70,8600,4,15.7
1469,Tom Brady,NE,5,IND,29.54,QB,19.0,19.74,8600,5,17.0
1837,Tom Brady,NE,6,KC,21.80,QB,28.0,22.23,8800,6,19.5
2167,Tom Brady,NE,7,CHI,22.68,QB,11.0,17.82,8700,7,19.9
2528,Tom Brady,NE,8,BUF,13.76,QB,4.0,19.29,8600,8,20.3
2830,Tom Brady,NE,9,GB,15.66,QB,19.0,21.58,8600,9,19.5
3162,Tom Brady,NE,10,TEN,11.26,QB,6.0,19.40,8300,10,19.1


In [68]:
betting_data = pd.read_excel('Data/clean_nfl_historic_betting_data_Home.xlsx')

In [69]:
betting_data_2 = pd.read_excel('Data/clean_nfl_historic_betting_data_Away.xlsx')

In [70]:
betting_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4039 entries, 0 to 4038
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               4039 non-null   datetime64[ns]
 1   Team               4039 non-null   object        
 2   Line Close         1903 non-null   float64       
 3   Total Score Close  1903 non-null   float64       
 4   Team Proj          1903 non-null   float64       
 5   Week               1903 non-null   float64       
 6   Home / Away        1903 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 221.0+ KB


In [71]:
betting_data = pd.concat([betting_data, betting_data_2])

In [72]:
betting_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8078 entries, 0 to 4038
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               8078 non-null   datetime64[ns]
 1   Team               8078 non-null   object        
 2   Line Close         3806 non-null   float64       
 3   Total Score Close  3806 non-null   float64       
 4   Team Proj          3806 non-null   float64       
 5   Week               3806 non-null   float64       
 6   Home / Away        3806 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 504.9+ KB


In [73]:
betting_data['Date'] = betting_data['Date'].dt.year

In [74]:
betting_data = betting_data[betting_data['Date'] == 2018]

In [75]:
betting_data = betting_data[betting_data['Week'] != 0]

In [76]:
betting_data

,Date,Team,Line Close,Total Score Close,Team Proj,Week,Home / Away
579,2018,TEN,4.5,42.5,16.75,17.0,Home
580,2018,BAL,-7.0,40.5,27.25,17.0,Home
581,2018,DEN,6.5,43.5,15.25,17.0,Home
582,2018,LAR,-10.5,49.5,35.25,17.0,Home
583,2018,MIN,-6.0,39.5,25.75,17.0,Home
...,...,...,...,...,...,...,...
830,2018,SF,6.0,46.5,17.25,1.0,Away
831,2018,HOU,6.5,49.5,18.25,1.0,Away
832,2018,TB,10.0,49.5,14.75,1.0,Away
833,2018,JAX,-3.0,42.5,24.25,1.0,Away


In [77]:
offense_2018 = offense_2018.merge(betting_data, how='inner', on=['Team', 'Week'])

In [78]:
offense_2018[offense_2018['Week'] == 12].head(50)

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG,Date,Line Close,Total Score Close,Team Proj,Home / Away
3704,Christian McCaffrey,CAR,12,SEA,41.20,RB,13.0,17.66,8400,11,18.9,2018,-3.0,46.0,26.00,Home
3705,Cam Newton,CAR,12,SEA,23.54,QB,3.0,20.80,8600,11,23.5,2018,-3.0,46.0,26.00,Home
3706,Curtis Samuel,CAR,12,SEA,11.20,WR,20.0,7.00,5300,8,8.4,2018,-3.0,46.0,26.00,Home
3707,Jarius Wright,CAR,12,SEA,3.50,WR,20.0,3.39,4700,11,4.7,2018,-3.0,46.0,26.00,Home
3708,Greg Olsen,CAR,12,SEA,2.10,TE,5.0,8.81,6500,8,9.0,2018,-3.0,46.0,26.00,Home
3709,Damiere Byrd,CAR,12,SEA,0.00,WR,20.0,0.68,4500,8,0.2,2018,-3.0,46.0,26.00,Home
3710,Torrey Smith,CAR,12,SEA,0.00,WR,20.0,5.03,5300,7,6.5,2018,-3.0,46.0,26.00,Home
3711,Cameron Artis-Payne,CAR,12,SEA,0.00,RB,13.0,2.25,4500,4,0.0,2018,-3.0,46.0,26.00,Home
3712,Chris Manhertz,CAR,12,SEA,0.00,TE,5.0,0.15,4000,11,0.1,2018,-3.0,46.0,26.00,Home
3713,Alex Armah,CAR,12,SEA,0.00,RB,13.0,0.38,4500,11,1.3,2018,-3.0,46.0,26.00,Home


## Merge 2017 and 2018

In [79]:
merged_df = pd.concat([offense_2017, offense_2018]).reset_index(drop=True)

In [80]:
merged_df

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG,Date,Line Close,Total Score Close,Team Proj,Home / Away
0,Kareem Hunt,KC,1,NE,40.60,RB,3.0,10.73,6400,1,22.9,2017,8.0,47.5,15.75,Away
1,Alex Smith,KC,1,NE,31.02,QB,11.0,14.69,6900,1,21.9,2017,8.0,47.5,15.75,Away
2,Tyreek Hill,KC,1,NE,19.80,WR,9.0,10.48,6700,1,13.8,2017,8.0,47.5,15.75,Away
3,Charcandrick West,KC,1,NE,8.50,RB,3.0,4.27,5000,1,4.4,2017,8.0,47.5,15.75,Away
4,Demetrius Harris,KC,1,NE,7.50,TE,6.0,2.03,4500,1,2.3,2017,8.0,47.5,15.75,Away
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11017,Adrian Peterson,WAS,17,PHI,0.00,RB,14.0,10.38,0,15,11.8,2018,6.0,43.0,15.50,Home
11018,Dustin Hopkins,WAS,17,PHI,0.00,K,30.0,8.11,0,1,NaN,2018,6.0,43.0,15.50,Home
11019,Jamaal Williams,GB,17,DET,3.50,RB,22.0,14.64,0,16,6.5,2018,-7.5,44.5,29.75,Home
11020,Aaron Rodgers,GB,17,DET,1.04,QB,21.0,20.53,0,16,20.9,2018,-7.5,44.5,29.75,Home


In [81]:
merged_df[merged_df['Salary'] == 0]

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG,Date,Line Close,Total Score Close,Team Proj,Home / Away
10892,Josh Allen,BUF,17,MIA,41.46,QB,16.0,17.76,0,12,16.2,2018,-5.5,39.5,25.25,Home
10893,Zay Jones,BUF,17,MIA,24.30,WR,14.0,8.44,0,16,7.5,2018,-5.5,39.5,25.25,Home
10894,LeSean McCoy,BUF,17,MIA,11.40,RB,29.0,8.11,0,14,7.6,2018,-5.5,39.5,25.25,Home
10895,Dak Prescott,DAL,17,NYG,33.48,QB,11.0,6.00,0,16,17.4,2018,7.0,38.5,12.25,Away
10896,Brett Maher,DAL,17,NYG,4.00,K,31.0,7.54,0,1,NaN,2018,7.0,38.5,12.25,Away
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11017,Adrian Peterson,WAS,17,PHI,0.00,RB,14.0,10.38,0,15,11.8,2018,6.0,43.0,15.50,Home
11018,Dustin Hopkins,WAS,17,PHI,0.00,K,30.0,8.11,0,1,NaN,2018,6.0,43.0,15.50,Home
11019,Jamaal Williams,GB,17,DET,3.50,RB,22.0,14.64,0,16,6.5,2018,-7.5,44.5,29.75,Home
11020,Aaron Rodgers,GB,17,DET,1.04,QB,21.0,20.53,0,16,20.9,2018,-7.5,44.5,29.75,Home


In [82]:
merged_df.to_csv('Data/clean_stats.csv')

#### Adding 2019

In [86]:
stats_list = []
for i in range(1,18):
    name = 'Data/Historic_Stats/2019_' + str(i) + '.csv'
    stats_list.append(name)

stats_list

['Data/Historic_Stats/2019_1.csv',
 'Data/Historic_Stats/2019_2.csv',
 'Data/Historic_Stats/2019_3.csv',
 'Data/Historic_Stats/2019_4.csv',
 'Data/Historic_Stats/2019_5.csv',
 'Data/Historic_Stats/2019_6.csv',
 'Data/Historic_Stats/2019_7.csv',
 'Data/Historic_Stats/2019_8.csv',
 'Data/Historic_Stats/2019_9.csv',
 'Data/Historic_Stats/2019_10.csv',
 'Data/Historic_Stats/2019_11.csv',
 'Data/Historic_Stats/2019_12.csv',
 'Data/Historic_Stats/2019_13.csv',
 'Data/Historic_Stats/2019_14.csv',
 'Data/Historic_Stats/2019_15.csv',
 'Data/Historic_Stats/2019_16.csv',
 'Data/Historic_Stats/2019_17.csv']

In [87]:
stats_2019 = pd.DataFrame()
for index, item in enumerate(stats_list):
    df = pd.read_csv(item)
    df = df[['Name', 'Team', 'Week', 'Opponent', 'FantasyPointsPerGameFanDuel', 'FantasyPointsFanDuel']]
    df = df.rename(columns={'FantasyPointsPerGameFanDuel':'FPPPG', 'FantasyPointsFanDuel':'Points_Scored'})
    stats_2019 = stats_2019.append(df)


In [88]:
stats_2019

,Name,Team,Week,Opponent,FPPPG,Points_Scored
0,Sammy Watkins,KC,1,JAX,42.3,42.30
1,Christian McCaffrey,CAR,1,LAR,37.9,37.90
2,Austin Ekeler,LAC,1,IND,36.4,36.40
3,Lamar Jackson,BAL,1,MIA,33.6,33.56
4,Dak Prescott,DAL,1,NYG,33.4,33.40
...,...,...,...,...,...,...
1185,Chase Edmonds,ARI,17,LAR,-0.1,-0.10
1186,Diontae Spencer,DEN,17,LV,-0.8,-0.80
1187,Will Grier,CAR,17,NO,-2.3,-2.34
1188,Carolina Panthers,CAR,17,NO,-3.0,-3.00


In [90]:
proj_list = []
for i in range(1,18):
    name = 'Data/Historic_Proj/2019_' + str(i) + '.csv'
    proj_list.append(name)
    
proj_list

['Data/Historic_Proj/2019_1.csv',
 'Data/Historic_Proj/2019_2.csv',
 'Data/Historic_Proj/2019_3.csv',
 'Data/Historic_Proj/2019_4.csv',
 'Data/Historic_Proj/2019_5.csv',
 'Data/Historic_Proj/2019_6.csv',
 'Data/Historic_Proj/2019_7.csv',
 'Data/Historic_Proj/2019_8.csv',
 'Data/Historic_Proj/2019_9.csv',
 'Data/Historic_Proj/2019_10.csv',
 'Data/Historic_Proj/2019_11.csv',
 'Data/Historic_Proj/2019_12.csv',
 'Data/Historic_Proj/2019_13.csv',
 'Data/Historic_Proj/2019_14.csv',
 'Data/Historic_Proj/2019_15.csv',
 'Data/Historic_Proj/2019_16.csv',
 'Data/Historic_Proj/2019_17.csv']

In [91]:
projections_2019 = pd.DataFrame()
for item in proj_list:
    df = pd.read_csv(item)
    df = df[['Name', 'Position', 'Week', 'Team','UpcomingOpponentPositionRank', 'FantasyPointsFanDuel', 'OperatorSalary']]
    df = df.rename(columns={'UpcomingOpponentPositionRank':'OPRNK', 'FantasyPointsFanDuel': 'Projected', 'OperatorSalary': 'Salary'})
    projections_2019 = projections_2019.append(df)

In [92]:
projections_2019

,Name,Position,Week,Team,OPRNK,Projected,Salary
0,Saquon Barkley,RB,1,NYG,8.0,18.45,9200
1,Ezekiel Elliott,RB,1,DAL,25.0,14.89,9100
2,DeAndre Hopkins,WR,1,HOU,32.0,16.29,9000
3,Christian McCaffrey,RB,1,CAR,15.0,18.80,8900
4,Patrick Mahomes,QB,1,KC,9.0,21.07,8700
...,...,...,...,...,...,...,...
649,Los Angeles Chargers,DST,17,LAC,NaN,5.20,3200
650,Washington Football Team,DST,17,WAS,NaN,4.30,3100
651,Detroit Lions,DST,17,DET,NaN,4.90,3000
652,Carolina Panthers,DST,17,CAR,NaN,4.70,3000


In [93]:
data_2019 = stats_2019.merge(projections_2019, how='inner', on=['Name', 'Week', 'Team'])

In [94]:
offense_2019 = data_2019[data_2019['Position'] != 'DST']

In [95]:
offense_2019 = offense_2019.drop_duplicates()

In [96]:
offense_2019['Name'].value_counts()

Emmanuel Sanders    17
Jarius Wright       16
Hayden Hurst        16
Russell Gage        16
Nick Boyle          16
                    ..
Chad Williams        1
Devin Funchess       1
Kendall Blanton      1
Alex Ellis           1
Patrick Ricard       1
Name: Name, Length: 530, dtype: int64

In [97]:
grouped = offense_2019.groupby(['Name', 'Week']).sum()

In [98]:
grouped['Games_Played'] = 1

In [99]:
new_df = grouped.groupby(level=0).cumsum().reset_index()

In [100]:
new_df['FP_per_game'] = round(new_df['Points_Scored'] / new_df['Games_Played'], 1)

In [101]:
new_df = new_df[['Name', 'Week', 'FP_per_game', 'Games_Played']]

In [102]:
new_df['Test'] = new_df.groupby('Name')['FP_per_game'].shift()

In [103]:
new_df['Test_2'] = round(new_df.groupby('Name')['Test'].transform(lambda x: x.fillna(x.mean())), 1)

In [104]:
new_df['FPPG'] = new_df['Test_2']

In [105]:
new_df = new_df.drop(['Test', 'Test_2', 'FP_per_game'], axis=1)

In [106]:
new_df

,Name,Week,Games_Played,FPPG
0,A.J. Brown,1,1,8.9
1,A.J. Brown,2,2,11.5
2,A.J. Brown,3,3,7.8
3,A.J. Brown,4,4,5.5
4,A.J. Brown,5,5,9.8
...,...,...,...,...
5642,Zay Jones,13,11,2.5
5643,Zay Jones,14,12,2.5
5644,Zay Jones,15,13,2.4
5645,Zay Jones,16,14,2.3


In [107]:
offense_2019 = pd.merge(offense_2019, new_df, how='inner', on=['Name', 'Week'])

In [108]:
offense_2019 = offense_2019.drop('FPPPG', axis=1)

In [110]:
offense_2019[offense_2019['Name'] == 'Emmanuel Sanders']

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG
49,Emmanuel Sanders,DEN,1,LV,17.1,WR,24.0,9.38,5600,1,11.4
376,Emmanuel Sanders,DEN,2,CHI,23.3,WR,3.0,9.55,5600,2,17.1
918,Emmanuel Sanders,DEN,3,GB,2.0,WR,8.0,11.69,5700,3,20.2
1124,Emmanuel Sanders,DEN,4,JAX,12.9,WR,19.0,9.75,5500,4,14.1
1586,Emmanuel Sanders,DEN,5,LAC,1.4,WR,15.0,11.89,5600,5,13.8
1941,Emmanuel Sanders,DEN,6,TEN,0.5,WR,10.0,10.65,5500,6,11.3
2113,Emmanuel Sanders,DEN,7,KC,8.5,WR,13.0,9.45,5500,7,9.5
2427,Emmanuel Sanders,SF,8,CAR,10.5,WR,20.0,8.00,5400,8,9.4
2707,Emmanuel Sanders,SF,9,ARI,20.7,WR,14.0,9.79,5800,9,9.5
3138,Emmanuel Sanders,SF,10,SEA,3.4,WR,15.0,11.55,6400,10,10.8


In [111]:
betting_data = pd.read_excel('Data/clean_nfl_historic_betting_data_Home.xlsx')

In [112]:
betting_data_2 = pd.read_excel('Data/clean_nfl_historic_betting_data_Away.xlsx')

In [113]:
betting_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4039 entries, 0 to 4038
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               4039 non-null   datetime64[ns]
 1   Team               4039 non-null   object        
 2   Line Close         1903 non-null   float64       
 3   Total Score Close  1903 non-null   float64       
 4   Team Proj          1903 non-null   float64       
 5   Week               1903 non-null   float64       
 6   Home / Away        1903 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 221.0+ KB


In [114]:
betting_data = pd.concat([betting_data, betting_data_2])

In [115]:
betting_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8078 entries, 0 to 4038
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               8078 non-null   datetime64[ns]
 1   Team               8078 non-null   object        
 2   Line Close         3806 non-null   float64       
 3   Total Score Close  3806 non-null   float64       
 4   Team Proj          3806 non-null   float64       
 5   Week               3806 non-null   float64       
 6   Home / Away        3806 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 504.9+ KB


In [116]:
betting_data['Date'] = betting_data['Date'].dt.year

In [117]:
betting_data = betting_data[betting_data['Date'] == 2019]

In [118]:
betting_data = betting_data[betting_data['Week'] != 0]

In [119]:
betting_data

,Date,Team,Line Close,Total Score Close,Team Proj,Week,Home / Away
312,2019,SEA,3.5,47.0,20.00,17.0,Home
313,2019,DEN,-4.5,41.0,25.00,17.0,Home
314,2019,LAR,-7.0,45.5,29.75,17.0,Home
315,2019,BAL,1.0,35.0,16.50,17.0,Home
316,2019,BUF,1.0,37.0,17.50,17.0,Home
...,...,...,...,...,...,...,...
563,2019,BAL,-7.0,40.5,27.25,1.0,Away
564,2019,ATL,3.5,47.0,20.00,1.0,Away
565,2019,BUF,2.5,41.0,18.00,1.0,Away
566,2019,WAS,10.0,44.5,12.25,1.0,Away


In [120]:
offense_2019 = offense_2019.merge(betting_data, how='inner', on=['Team', 'Week'])

In [121]:
offense_2019[offense_2019['Week'] == 12].head(50)

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG,Date,Line Close,Total Score Close,Team Proj,Home / Away
3594,Lamar Jackson,BAL,12,LAR,36.26,QB,10.0,24.12,9000,11,27.5,2019,-3.5,47.0,27.00,Away
3595,Marquise Brown,BAL,12,LAR,18.70,WR,9.0,8.71,5600,9,10.8,2019,-3.5,47.0,27.00,Away
3596,Miles Boykin,BAL,12,LAR,6.40,WR,9.0,3.28,4700,11,3.0,2019,-3.5,47.0,27.00,Away
3597,Gus Edwards,BAL,12,LAR,5.50,RB,13.0,3.92,5400,11,5.6,2019,-3.5,47.0,27.00,Away
3598,Mark Andrews,BAL,12,LAR,5.50,TE,13.0,10.52,6900,11,11.8,2019,-3.5,47.0,27.00,Away
3599,Seth Roberts,BAL,12,LAR,5.40,WR,9.0,2.47,4700,11,2.4,2019,-3.5,47.0,27.00,Away
3600,Justice Hill,BAL,12,LAR,2.70,RB,13.0,1.36,4500,11,1.6,2019,-3.5,47.0,27.00,Away
3601,Robert Griffin III,BAL,12,LAR,1.26,QB,10.0,0.96,6000,4,2.8,2019,-3.5,47.0,27.00,Away
3602,Nick Boyle,BAL,12,LAR,1.20,TE,13.0,3.91,4800,11,4.4,2019,-3.5,47.0,27.00,Away
3603,Chris Moore,BAL,12,LAR,0.00,WR,9.0,0.38,4600,9,0.4,2019,-3.5,47.0,27.00,Away


In [122]:
merged_df[merged_df['Salary'] == 0]

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG,Date,Line Close,Total Score Close,Team Proj,Home / Away
10892,Josh Allen,BUF,17,MIA,41.46,QB,16.0,17.76,0,12,16.2,2018,-5.5,39.5,25.25,Home
10893,Zay Jones,BUF,17,MIA,24.30,WR,14.0,8.44,0,16,7.5,2018,-5.5,39.5,25.25,Home
10894,LeSean McCoy,BUF,17,MIA,11.40,RB,29.0,8.11,0,14,7.6,2018,-5.5,39.5,25.25,Home
10895,Dak Prescott,DAL,17,NYG,33.48,QB,11.0,6.00,0,16,17.4,2018,7.0,38.5,12.25,Away
10896,Brett Maher,DAL,17,NYG,4.00,K,31.0,7.54,0,1,NaN,2018,7.0,38.5,12.25,Away
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11017,Adrian Peterson,WAS,17,PHI,0.00,RB,14.0,10.38,0,15,11.8,2018,6.0,43.0,15.50,Home
11018,Dustin Hopkins,WAS,17,PHI,0.00,K,30.0,8.11,0,1,NaN,2018,6.0,43.0,15.50,Home
11019,Jamaal Williams,GB,17,DET,3.50,RB,22.0,14.64,0,16,6.5,2018,-7.5,44.5,29.75,Home
11020,Aaron Rodgers,GB,17,DET,1.04,QB,21.0,20.53,0,16,20.9,2018,-7.5,44.5,29.75,Home


In [123]:
merged_df = pd.concat([merged_df, offense_2019]).reset_index(drop=True)

In [124]:
merged_df

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG,Date,Line Close,Total Score Close,Team Proj,Home / Away
0,Kareem Hunt,KC,1,NE,40.60,RB,3.0,10.73,6400,1,22.9,2017,8.0,47.5,15.75,Away
1,Alex Smith,KC,1,NE,31.02,QB,11.0,14.69,6900,1,21.9,2017,8.0,47.5,15.75,Away
2,Tyreek Hill,KC,1,NE,19.80,WR,9.0,10.48,6700,1,13.8,2017,8.0,47.5,15.75,Away
3,Charcandrick West,KC,1,NE,8.50,RB,3.0,4.27,5000,1,4.4,2017,8.0,47.5,15.75,Away
4,Demetrius Harris,KC,1,NE,7.50,TE,6.0,2.03,4500,1,2.3,2017,8.0,47.5,15.75,Away
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16665,Jeremy Sprinkle,WAS,17,DAL,3.30,TE,25.0,3.62,4500,16,2.7,2019,12.5,47.5,11.25,Away
16666,Cam Sims,WAS,17,DAL,1.70,WR,6.0,3.76,4500,7,0.3,2019,12.5,47.5,11.25,Away
16667,Chris Thompson,WAS,17,DAL,1.60,RB,13.0,7.29,5100,11,7.1,2019,12.5,47.5,11.25,Away
16668,Wendell Smallwood,WAS,17,DAL,0.50,RB,13.0,1.30,4500,15,1.3,2019,12.5,47.5,11.25,Away


#### Add 2020

In [125]:
stats_list = []
for i in range(1,18):
    name = 'Data/Historic_Stats/2020_' + str(i) + '.csv'
    stats_list.append(name)

stats_list

['Data/Historic_Stats/2020_1.csv',
 'Data/Historic_Stats/2020_2.csv',
 'Data/Historic_Stats/2020_3.csv',
 'Data/Historic_Stats/2020_4.csv',
 'Data/Historic_Stats/2020_5.csv',
 'Data/Historic_Stats/2020_6.csv',
 'Data/Historic_Stats/2020_7.csv',
 'Data/Historic_Stats/2020_8.csv',
 'Data/Historic_Stats/2020_9.csv',
 'Data/Historic_Stats/2020_10.csv',
 'Data/Historic_Stats/2020_11.csv',
 'Data/Historic_Stats/2020_12.csv',
 'Data/Historic_Stats/2020_13.csv',
 'Data/Historic_Stats/2020_14.csv',
 'Data/Historic_Stats/2020_15.csv',
 'Data/Historic_Stats/2020_16.csv',
 'Data/Historic_Stats/2020_17.csv']

In [126]:
stats_2020 = pd.DataFrame()
for index, item in enumerate(stats_list):
    df = pd.read_csv(item)
    df = df[['Name', 'Team', 'Week', 'Opponent', 'FantasyPointsPerGameFanDuel', 'FantasyPointsFanDuel']]
    df = df.rename(columns={'FantasyPointsPerGameFanDuel':'FPPPG', 'FantasyPointsFanDuel':'Points_Scored'})
    stats_2020 = stats_2020.append(df)


In [127]:
stats_2020

,Name,Team,Week,Opponent,FPPPG,Points_Scored
0,Davante Adams,GB,1,MIN,34.6,34.60
1,Josh Jacobs,LV,1,CAR,33.9,33.90
2,Russell Wilson,SEA,1,ATL,31.8,31.78
3,Aaron Rodgers,GB,1,MIN,30.8,30.76
4,Calvin Ridley,ATL,1,SEA,29.4,29.40
...,...,...,...,...,...,...
1221,Chicago Bears,CHI,17,GB,-1.0,-1.00
1222,Atlanta Falcons,ATL,17,TB,-1.0,-1.00
1223,Mack Hollins,MIA,17,BUF,-1.2,-1.20
1224,Jamal Agnew,DET,17,MIN,-2.0,-2.00


In [128]:
proj_list = []
for i in range(1,18):
    name = 'Data/Historic_Proj/2020_' + str(i) + '.csv'
    proj_list.append(name)
    
proj_list

['Data/Historic_Proj/2020_1.csv',
 'Data/Historic_Proj/2020_2.csv',
 'Data/Historic_Proj/2020_3.csv',
 'Data/Historic_Proj/2020_4.csv',
 'Data/Historic_Proj/2020_5.csv',
 'Data/Historic_Proj/2020_6.csv',
 'Data/Historic_Proj/2020_7.csv',
 'Data/Historic_Proj/2020_8.csv',
 'Data/Historic_Proj/2020_9.csv',
 'Data/Historic_Proj/2020_10.csv',
 'Data/Historic_Proj/2020_11.csv',
 'Data/Historic_Proj/2020_12.csv',
 'Data/Historic_Proj/2020_13.csv',
 'Data/Historic_Proj/2020_14.csv',
 'Data/Historic_Proj/2020_15.csv',
 'Data/Historic_Proj/2020_16.csv',
 'Data/Historic_Proj/2020_17.csv']

In [129]:
projections_2020 = pd.DataFrame()
for item in proj_list:
    df = pd.read_csv(item)
    df = df[['Name', 'Position', 'Week', 'Team','UpcomingOpponentPositionRank', 'FantasyPointsFanDuel', 'OperatorSalary']]
    df = df.rename(columns={'UpcomingOpponentPositionRank':'OPRNK', 'FantasyPointsFanDuel': 'Projected', 'OperatorSalary': 'Salary'})
    projections_2020 = projections_2020.append(df)

In [130]:
projections_2020

,Name,Position,Week,Team,OPRNK,Projected,Salary
0,Christian McCaffrey,RB,1,CAR,19.0,20.81,10000
1,Patrick Mahomes,QB,1,KC,29.0,23.76,9500
2,Lamar Jackson,QB,1,BAL,21.0,24.16,9300
3,Saquon Barkley,RB,1,NYG,5.0,16.51,9000
4,Michael Thomas,WR,1,NO,32.0,16.49,8800
...,...,...,...,...,...,...,...
826,Carolina Panthers,DST,17,CAR,NaN,4.80,3300
827,Atlanta Falcons,DST,17,ATL,NaN,4.00,3300
828,Houston Texans,DST,17,HOU,NaN,4.00,3200
829,Jacksonville Jaguars,DST,17,JAX,NaN,3.60,3100


In [131]:
data_2020 = stats_2020.merge(projections_2020, how='inner', on=['Name', 'Week', 'Team'])

In [132]:
offense_2020 = data_2020[data_2020['Position'] != 'DST']

In [133]:
offense_2020 = offense_2020.drop_duplicates()

In [134]:
offense_2020['Name'].value_counts()

Zay Jones           16
Ryan Tannehill      16
Darren Fells        16
Kerryon Johnson     16
Jalen Guyton        16
                    ..
Tommy Stevens        1
Kendall Hinton       1
Tony Brown           1
Antonio Williams     1
Blake Jarwin         1
Name: Name, Length: 629, dtype: int64

In [135]:
grouped = offense_2020.groupby(['Name', 'Week']).sum()

In [136]:
grouped['Games_Played'] = 1

In [137]:
new_df = grouped.groupby(level=0).cumsum().reset_index()

In [138]:
new_df['FP_per_game'] = round(new_df['Points_Scored'] / new_df['Games_Played'], 1)

In [139]:
new_df = new_df[['Name', 'Week', 'FP_per_game', 'Games_Played']]

In [140]:
new_df['Test'] = new_df.groupby('Name')['FP_per_game'].shift()

In [141]:
new_df['Test_2'] = round(new_df.groupby('Name')['Test'].transform(lambda x: x.fillna(x.mean())), 1)

In [142]:
new_df['FPPG'] = new_df['Test_2']

In [143]:
new_df = new_df.drop(['Test', 'Test_2', 'FP_per_game'], axis=1)

In [144]:
new_df

,Name,Week,Games_Played,FPPG
0,A.J. Brown,1,1,14.3
1,A.J. Brown,5,2,6.4
2,A.J. Brown,6,3,12.0
3,A.J. Brown,7,4,14.8
4,A.J. Brown,8,5,17.2
...,...,...,...,...
6517,Zay Jones,13,12,2.1
6518,Zay Jones,14,13,1.9
6519,Zay Jones,15,14,1.8
6520,Zay Jones,16,15,1.8


In [145]:
offense_2020 = pd.merge(offense_2020, new_df, how='inner', on=['Name', 'Week'])

In [146]:
offense_2020 = offense_2020.drop('FPPPG', axis=1)

In [147]:
offense_2020[offense_2020['Name'] == 'Tom Brady']

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG
19,Tom Brady,TB,1,NO,22.46,QB,20.0,17.98,7500,1,20.6
521,Tom Brady,TB,2,CAR,9.68,QB,9.0,18.82,7600,2,22.5
839,Tom Brady,TB,3,DEN,23.88,QB,15.0,17.99,7300,3,16.1
1235,Tom Brady,TB,4,LAC,33.46,QB,18.0,19.23,7300,4,18.7
1651,Tom Brady,TB,5,CHI,14.12,QB,2.0,17.90,7500,5,22.4
1993,Tom Brady,TB,6,GB,14.64,QB,13.0,19.70,7500,6,20.7
2313,Tom Brady,TB,7,LV,36.86,QB,23.0,19.57,7400,7,19.7
2685,Tom Brady,TB,8,NYG,19.06,QB,11.0,19.77,7700,8,22.2
3179,Tom Brady,TB,9,NO,5.36,QB,27.0,20.27,7800,9,21.8
3372,Tom Brady,TB,10,CAR,31.84,QB,11.0,19.87,7800,10,19.9


In [148]:
betting_data = pd.read_excel('Data/clean_nfl_historic_betting_data_Home.xlsx')

In [149]:
betting_data_2 = pd.read_excel('Data/clean_nfl_historic_betting_data_Away.xlsx')

In [150]:
betting_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4039 entries, 0 to 4038
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               4039 non-null   datetime64[ns]
 1   Team               4039 non-null   object        
 2   Line Close         1903 non-null   float64       
 3   Total Score Close  1903 non-null   float64       
 4   Team Proj          1903 non-null   float64       
 5   Week               1903 non-null   float64       
 6   Home / Away        1903 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 221.0+ KB


In [151]:
betting_data = pd.concat([betting_data, betting_data_2])

In [152]:
betting_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8078 entries, 0 to 4038
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               8078 non-null   datetime64[ns]
 1   Team               8078 non-null   object        
 2   Line Close         3806 non-null   float64       
 3   Total Score Close  3806 non-null   float64       
 4   Team Proj          3806 non-null   float64       
 5   Week               3806 non-null   float64       
 6   Home / Away        3806 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 504.9+ KB


In [153]:
betting_data['Date'] = betting_data['Date'].dt.year

In [156]:
betting_data = betting_data[betting_data['Date'] == 2020]

In [157]:
betting_data = betting_data[betting_data['Week'] != 0]

In [158]:
betting_data

,Date,Team,Line Close,Total Score Close,Team Proj,Week,Home / Away
61,2020,NE,7.0,47.0,16.50,16.0,Home
62,2020,GB,-3.0,52.5,29.25,16.0,Home
63,2020,DAL,3.0,50.5,22.25,16.0,Home
64,2020,LAC,-2.0,47.5,25.75,16.0,Home
65,2020,SEA,-1.5,48.0,25.50,16.0,Home
...,...,...,...,...,...,...,...
296,2020,IND,-8.0,45.0,30.50,1.0,Away
297,2020,GB,2.5,44.5,19.75,1.0,Away
298,2020,MIA,7.0,41.5,13.75,1.0,Away
299,2020,PHI,-5.0,42.0,26.00,1.0,Away


In [159]:
offense_2020 = offense_2020.merge(betting_data, how='inner', on=['Team', 'Week'])

In [160]:
offense_2020[offense_2020['Week'] == 12].head(50)

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG,Date,Line Close,Total Score Close,Team Proj,Home / Away
4093,Tyreek Hill,KC,12,TB,51.40,WR,13.0,16.09,8600,10,17.8,2020,-3.5,56.0,31.50,Away
4094,Patrick Mahomes,KC,12,TB,31.28,QB,8.0,24.17,9000,10,26.9,2020,-3.5,56.0,31.50,Away
4095,Travis Kelce,KC,12,TB,12.20,TE,16.0,15.31,8000,10,17.1,2020,-3.5,56.0,31.50,Away
4096,Demarcus Robinson,KC,12,TB,6.10,WR,13.0,4.61,5300,10,5.5,2020,-3.5,56.0,31.50,Away
4097,Sammy Watkins,KC,12,TB,5.80,WR,13.0,8.48,5600,5,9.7,2020,-3.5,56.0,31.50,Away
4098,Clyde Edwards-Helaire,KC,12,TB,4.40,RB,3.0,11.71,6700,10,13.6,2020,-3.5,56.0,31.50,Away
4099,Le'Veon Bell,KC,12,TB,4.20,RB,3.0,5.23,5600,7,5.5,2020,-3.5,56.0,31.50,Away
4100,Mecole Hardman,KC,12,TB,3.80,WR,13.0,6.63,5500,10,7.2,2020,-3.5,56.0,31.50,Away
4101,Deon Yelder,KC,12,TB,0.70,TE,16.0,0.36,4000,8,0.1,2020,-3.5,56.0,31.50,Away
4102,Nick Keizer,KC,12,TB,0.00,TE,16.0,1.28,4100,9,0.7,2020,-3.5,56.0,31.50,Away


In [161]:
merged_df[merged_df['Salary'] == 0]

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG,Date,Line Close,Total Score Close,Team Proj,Home / Away
10892,Josh Allen,BUF,17,MIA,41.46,QB,16.0,17.76,0,12,16.2,2018,-5.5,39.5,25.25,Home
10893,Zay Jones,BUF,17,MIA,24.30,WR,14.0,8.44,0,16,7.5,2018,-5.5,39.5,25.25,Home
10894,LeSean McCoy,BUF,17,MIA,11.40,RB,29.0,8.11,0,14,7.6,2018,-5.5,39.5,25.25,Home
10895,Dak Prescott,DAL,17,NYG,33.48,QB,11.0,6.00,0,16,17.4,2018,7.0,38.5,12.25,Away
10896,Brett Maher,DAL,17,NYG,4.00,K,31.0,7.54,0,1,NaN,2018,7.0,38.5,12.25,Away
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11017,Adrian Peterson,WAS,17,PHI,0.00,RB,14.0,10.38,0,15,11.8,2018,6.0,43.0,15.50,Home
11018,Dustin Hopkins,WAS,17,PHI,0.00,K,30.0,8.11,0,1,NaN,2018,6.0,43.0,15.50,Home
11019,Jamaal Williams,GB,17,DET,3.50,RB,22.0,14.64,0,16,6.5,2018,-7.5,44.5,29.75,Home
11020,Aaron Rodgers,GB,17,DET,1.04,QB,21.0,20.53,0,16,20.9,2018,-7.5,44.5,29.75,Home


In [162]:
merged_df = pd.concat([merged_df, offense_2020]).reset_index(drop=True)

In [165]:
merged_df = merged_df[merged_df['Projected'] != 0]

In [166]:
merged_df

,Name,Team,Week,Opponent,Points_Scored,Position,OPRNK,Projected,Salary,Games_Played,FPPG,Date,Line Close,Total Score Close,Team Proj,Home / Away
0,Kareem Hunt,KC,1,NE,40.60,RB,3.0,10.73,6400,1,22.9,2017,8.0,47.5,15.75,Away
1,Alex Smith,KC,1,NE,31.02,QB,11.0,14.69,6900,1,21.9,2017,8.0,47.5,15.75,Away
2,Tyreek Hill,KC,1,NE,19.80,WR,9.0,10.48,6700,1,13.8,2017,8.0,47.5,15.75,Away
3,Charcandrick West,KC,1,NE,8.50,RB,3.0,4.27,5000,1,4.4,2017,8.0,47.5,15.75,Away
4,Demetrius Harris,KC,1,NE,7.50,TE,6.0,2.03,4500,1,2.3,2017,8.0,47.5,15.75,Away
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22776,Quintez Cephus,DET,16,TB,1.40,WR,22.0,4.03,4800,12,3.9,2020,12.0,55.0,15.50,Home
22777,Adrian Peterson,DET,16,TB,1.00,RB,1.0,4.16,5200,15,7.5,2020,12.0,55.0,15.50,Home
22778,Matthew Stafford,DET,16,TB,0.68,QB,16.0,17.76,7400,15,17.6,2020,12.0,55.0,15.50,Home
22779,Jesse James,DET,16,TB,0.00,TE,23.0,1.44,4300,15,2.2,2020,12.0,55.0,15.50,Home


In [167]:
merged_df.to_csv('Data/clean_stats.csv')

### Clean up Projections Data

In [670]:
stats_2021 = pd.read_csv('Data/Historic_Stats/2021_1.csv')

In [671]:
stats_2021 = stats_2021[['Name', 'Team', 'Week', 'Opponent', 'FantasyPointsPerGameFanDuel', 'FantasyPointsFanDuel']]
stats_2021 = stats_2021.rename(columns={'FantasyPointsPerGameFanDuel':'FPPG', 'FantasyPointsFanDuel':'Points_Scored'})

In [672]:
stats_2021 = stats_2021[['Name', 'FPPG']]

In [673]:
stats_2021

,Name,FPPG
0,Kyler Murray,34.6
1,Patrick Mahomes,33.3
2,Amari Cooper,32.4
3,Tyreek Hill,31.6
4,Jared Goff,29.9
...,...,...
1243,Isaiah McKenzie,-0.3
1244,Rhamondre Stevenson,-0.4
1245,Ronald Jones II,-0.6
1246,Jacksonville Jaguars,-3.0


In [676]:
week_2_projections = pd.read_csv('Data/Historic_Proj/2021_2.csv')

In [679]:
week_2_projections = week_2_projections[['Name', 'Position','Opponent', 'Week', 'Team','UpcomingOpponentPositionRank', 'FantasyPointsFanDuel', 'OperatorSalary', 'FanShareProjectedOwnershipPercentage']]
week_2_projections = week_2_projections.rename(columns={'UpcomingOpponentPositionRank':'OPRNK', 'FantasyPointsFanDuel': 'Projected', 'OperatorSalary': 'Salary', 'FanShareProjectedOwnershipPercentage': 'Percentage' })

In [680]:
week_2_projections

,Name,Position,Opponent,Week,Team,OPRNK,Projected,Salary,Percentage
0,Christian McCaffrey,RB,NO,2,CAR,3.0,19.80,10000,0.1
1,Dalvin Cook,RB,ARI,2,MIN,10.0,18.83,9200,12.1
2,Alvin Kamara,RB,CAR,2,NO,5.0,19.90,8800,21.2
3,Kyler Murray,QB,MIN,2,ARI,14.0,25.14,8700,11.5
4,Russell Wilson,QB,TEN,2,SEA,32.0,23.34,8500,8.3
...,...,...,...,...,...,...,...,...,...
657,Minnesota Vikings,DST,ARI,2,MIN,NaN,7.00,3300,0.0
658,Jacksonville Jaguars,DST,DEN,2,JAX,NaN,5.90,3300,0.0
659,Tennessee Titans,DST,SEA,2,TEN,NaN,5.30,3200,0.0
660,Houston Texans,DST,CLE,2,HOU,NaN,4.20,3100,0.0


In [681]:
week_2_projections = week_2_projections.merge(stats_2021, how='inner', on=['Name'])

In [682]:
week_2_projections

,Name,Position,Opponent,Week,Team,OPRNK,Projected,Salary,Percentage,FPPG
0,Christian McCaffrey,RB,NO,2,CAR,3.0,19.80,10000,0.1,23.2
1,Dalvin Cook,RB,ARI,2,MIN,10.0,18.83,9200,12.1,17.4
2,Alvin Kamara,RB,CAR,2,NO,5.0,19.90,8800,21.2,16.6
3,Kyler Murray,QB,MIN,2,ARI,14.0,25.14,8700,11.5,34.6
4,Russell Wilson,QB,TEN,2,SEA,32.0,23.34,8500,8.3,27.1
...,...,...,...,...,...,...,...,...,...,...
372,Minnesota Vikings,DST,ARI,2,MIN,NaN,7.00,3300,0.0,5.0
373,Jacksonville Jaguars,DST,DEN,2,JAX,NaN,5.90,3300,0.0,-3.0
374,Tennessee Titans,DST,SEA,2,TEN,NaN,5.30,3200,0.0,0.0
375,Houston Texans,DST,CLE,2,HOU,NaN,4.20,3100,0.0,7.0


In [683]:
week_2_projections = week_2_projections.drop_duplicates()

In [684]:
week_2_projections

,Name,Position,Opponent,Week,Team,OPRNK,Projected,Salary,Percentage,FPPG
0,Christian McCaffrey,RB,NO,2,CAR,3.0,19.80,10000,0.1,23.2
1,Dalvin Cook,RB,ARI,2,MIN,10.0,18.83,9200,12.1,17.4
2,Alvin Kamara,RB,CAR,2,NO,5.0,19.90,8800,21.2,16.6
3,Kyler Murray,QB,MIN,2,ARI,14.0,25.14,8700,11.5,34.6
4,Russell Wilson,QB,TEN,2,SEA,32.0,23.34,8500,8.3,27.1
...,...,...,...,...,...,...,...,...,...,...
372,Minnesota Vikings,DST,ARI,2,MIN,NaN,7.00,3300,0.0,5.0
373,Jacksonville Jaguars,DST,DEN,2,JAX,NaN,5.90,3300,0.0,-3.0
374,Tennessee Titans,DST,SEA,2,TEN,NaN,5.30,3200,0.0,0.0
375,Houston Texans,DST,CLE,2,HOU,NaN,4.20,3100,0.0,7.0


In [685]:
betting_data = pd.read_excel('Data/clean_nfl_historic_betting_data_Home.xlsx')

In [686]:
betting_data_2 = pd.read_excel('Data/clean_nfl_historic_betting_data_Away.xlsx')

In [687]:
betting_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4039 entries, 0 to 4038
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               4039 non-null   datetime64[ns]
 1   Team               4039 non-null   object        
 2   Line Close         1903 non-null   float64       
 3   Total Score Close  1903 non-null   float64       
 4   Team Proj          1903 non-null   float64       
 5   Week               1903 non-null   float64       
 6   Home / Away        1903 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 221.0+ KB


In [688]:
betting_data = pd.concat([betting_data, betting_data_2])

In [689]:
betting_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8078 entries, 0 to 4038
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               8078 non-null   datetime64[ns]
 1   Team               8078 non-null   object        
 2   Line Close         3806 non-null   float64       
 3   Total Score Close  3806 non-null   float64       
 4   Team Proj          3806 non-null   float64       
 5   Week               3806 non-null   float64       
 6   Home / Away        3806 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 504.9+ KB


In [690]:
betting_data['Date'] = betting_data['Date'].dt.year

In [691]:
betting_data = betting_data[betting_data['Date'] == 2021]

In [692]:
betting_data = betting_data[betting_data['Week'] != 0]

In [693]:
betting_data

,Date,Team,Line Close,Total Score Close,Team Proj,Week,Home / Away
0,2021,GB,-10.5,49.5,35.25,2.0,Home
1,2021,BAL,2.5,55.5,25.25,2.0,Home
2,2021,LAC,-2.5,52.5,28.75,2.0,Home
3,2021,SEA,-5.5,52.5,31.75,2.0,Home
4,2021,TB,-12.5,52.5,38.75,2.0,Home
...,...,...,...,...,...,...,...
56,2021,TEN,-7.0,55.5,34.75,17.0,Away
57,2021,JAX,15.5,48.0,8.50,17.0,Away
58,2021,NYJ,3.0,41.0,17.50,17.0,Away
59,2021,DAL,-1.5,43.5,23.25,17.0,Away


In [694]:
week_2_projections = week_2_projections.merge(betting_data, how='inner', on=['Team', 'Week'])

In [695]:
week_2_projections

,Name,Position,Opponent,Week,Team,OPRNK,Projected,Salary,Percentage,FPPG,Date,Line Close,Total Score Close,Team Proj,Home / Away
0,Christian McCaffrey,RB,NO,2,CAR,3.0,19.80,10000,0.1,23.2,2021,3.5,46.5,19.75,Home
1,DJ Moore,WR,NO,2,CAR,4.0,12.15,6700,3.9,12.4,2021,3.5,46.5,19.75,Home
2,Sam Darnold,QB,NO,2,CAR,1.0,17.06,6600,1.0,19.1,2021,3.5,46.5,19.75,Home
3,Robby Anderson,WR,NO,2,CAR,4.0,9.91,6300,3.9,12.2,2021,3.5,46.5,19.75,Home
4,Terrace Marshall Jr.,WR,NO,2,CAR,4.0,6.16,5000,2.4,4.1,2021,3.5,46.5,19.75,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,Jeff Smith,WR,NE,2,NYJ,6.0,0.44,4500,0.0,0.0,2021,5.5,43.5,16.25,Home
373,Justin Hardee,DB,NE,2,NYJ,NaN,0.00,4500,NaN,0.0,2021,5.5,43.5,16.25,Home
374,Trevon Wesco,TE,NE,2,NYJ,3.0,0.48,4100,0.0,0.0,2021,5.5,43.5,16.25,Home
375,Daniel Brown,TE,NE,2,NYJ,3.0,0.14,4000,0.0,0.0,2021,5.5,43.5,16.25,Home


In [696]:
week_2_projections.to_csv('Data/week_2_projections.csv')